### Designing multiple finite state automata using inheritance in Python

CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
Fall 2023
***

Project 1 requires you to create a set of FSAs, one for each legal token type allowed in the _Datalog_ database language. Here are the first several token types from the project.

| Token Type    | Description           |  
| :--           | :--                   |  
| COMMA         | The ',' character     |
| PERIOD        | The '.' character     |
| Q_MARK        | The '?' character     |
| LEFT_PAREN    | The '(' character     |
| RIGHT_PAREN   | The ')' character     |
| COLON         | The ':' character     |
| COLON-DASH    | The string ":-"       |

Since we have to build a lot of FSAs, we'll use good software engineering practice and use inheritance.

***

We'll use the inheritance pattern from 

https://www.geeksforgeeks.org/inheritance-in-python/

which says the following:

"If you forget to invoke the __init__() of the parent class then its instance variables would not be available to the child [derived] class." 

The error produced by the inheritance patter will look like:

    Traceback (most recent call last):
    File "myfile.py", line 12, in 
        method_name
    AttributeError: 'Derived_class' object has no attribute 'method_name'



***
Let's approach this by copying the essential pieces of the FSA class from the _FSA_code_example_ notebook. Observe the following differences from the _FSA_code_example_ notebook:

 * I called the base class simply __FSA__ rather than __colonDash_FSA__
 * I passed the name of the FSA in as an argument to the constructor
 * The constructor leaves the set of accept states undefined.
 * The constructor also leaves the FSA name empty
 * The starting state is S0, but the body of the S0 method only raises an error.
 * No methods are defined for any other state.
 
I also removed the part of the method that tracked history since we won't use it in this tutorial.

Go ahead and execute the code below. That will define the base class, and we can "inherit from it" in later parts of the tutorial.

In [1]:
class FSA:
    """ FSA Base or Super class"""
    def __init__(self,name):
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state S0, S1, S2, Serr will be represented by its own method
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state = self.S0  # We'll always have the start state named S0
        self.accept_states = set()  # No accept states defined
        # Transition function: Each transition will be defined in the state methods
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string = ""      # Default empty input
        self.FSA_name = name
        self.num_chars_read = 0
            
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def S0(self):
        """ Every FSA must have a start state, and we'll always name 
        it S0. The method for the start state must be defined in the
        derived class since it's not defined here. """
        raise NotImplementedError()     # This line causes an error to 
                                        # occur if the child classes don't implement this method

    ############################
    # Public Manager Functions
    ############################
    def Run(self,input_string):
        ###############################################################
        # This function will be called to make the FSA execute
        # It records the input string,
        # sets the current state to the start state
        # and then calls a state method that returns the next state
        # Each state method accesses the input_string
        ###############################################################
            # Remember input_string
        self.input_string = input_string
            # Set current state to start state
        current_state = self.start_state
            # Call current state, which starts the FSA
        while self.num_chars_read < len(self.input_string):
            current_state = current_state()
            # Check whether the FSA ended in an accept state
        outcome = False 
        if current_state in self.accept_states: outcome = True # Accept if the FSA ended in an accept state
        return outcome
    def Reset(self):
        self.num_chars_read = 0
        self.history = []
    
    ############################
    # Public Getters and Setters
    ############################
    def get_Name(self): return self.FSA_name
    def set_Name(self,FSA_name): self.FSA_name = FSA_name
    
    ############################
    # Private Helper functions
    ############################
    def __getCurrentInput(self):  # The double underscore makes the method private
        current_input = self.input_string[self.num_chars_read]
        self.num_chars_read += 1
        return current_input


***
We can create a derived (or child) class from the FSA base class by doing three things. First, we include the name of the base class inside the parentheses of the derived class:

        class derived_class(base_class):

Second, we invoke the constructor of the base class from within the constructor of the derived class, which must be done or bad stuff happens:

        def __init__(self,name):
                base_class.__init__(self,name)
                # Rest of constructor

Third, you must define a method in the child_class for each unimplemented method in the parent_class. In our case, the unimplemented method in the parent class is __S0__.

Once these three things are done, you can add whatever other methods you want to the derived_class. I'll add the states that make the derived_class perform like the colonDash_FSA from the _FSA_code_example_ Jupyter notebook.

***
The code example that follows calls the constructor, which calls the construct of the base_class and defines an accept state. The code then defines the methods for each state in the colonDash finite state automaton. I just copied these methods straight from the _FSA_code_example_ Jupyter notebook (except I dropped the history tracking that we used to construct the trace).

In [2]:
class colonDash_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"colonDash_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S2) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self.__getCurrentInput()
        if current_input == ':': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self.__getCurrentInput()
        if current_input == '-': next_state = self.S2
        else: next_state = self.Serr
        return next_state
    def S2(self):
        current_input = self.__getCurrentInput()
        next_state = self.S2 # loop in state S2
        return next_state
    def Serr(self):
        current_input = self.__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state
    

I love how clean this code is. Let's run it and see if it works.

In [3]:
my_FSA = colonDash_FSA()
input_string = ":-"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")


AttributeError: 'colonDash_FSA' object has no attribute '_colonDash_FSA__getCurrentInput'

It failed! It took me a while to figure out why. Since I believe that using Chat GPT-like tools will be an important skill in programming, it makes sense to me that we learn to use it as a tool in a way that helps you learn and helps you program faster. 

Here's what Chat GPT said when I typed in the prompt _Explain this python error to me_ and then copied the error above.

    This error message is indicating that there is an attribute error occurring in the Python code. Specifically, it is telling us that the 'colonDash_FSA' object does not have the attribute '__getCurrentInput', which is being called in the 'S0' method on line 7 of the 'colonDash_FSA' class.

    This error message suggests that there may be an issue with the implementation of the 'colonDash_FSA' class. One possibility is that the '__getCurrentInput' method has not been properly defined within the class or that it is a private method and is being called incorrectly. Another possibility is that there may be a typo or other error in the code that is causing the attribute to not be found.

    To resolve this error, you should check the implementation of the 'colonDash_FSA' class and ensure that the '__getCurrentInput' method is properly defined and accessible to the 'S0' method. You should also check for any typos or other errors in the code that could be causing the issue.
***

That explanation helped a bit because it allowed me to focus on one property that I try to follow in my programming style: keep private methods private. 

The problem is that the __getCurrentInput method in the base_class is __private__. (In Python, private methods are preceded by a double underscore.) Since the method is private in the base_class we can't access it in the derived_class. 

A solution is described here: 
https://stackoverflow.com/questions/20261517/inheritance-of-private-and-protected-methods-in-python

Let's redefine the derived_class. Observe how every call to 

    self.__getCurrentInput()

is replaced by 

    self._FSA__getCurrentInput()

since FSA is the base_class. Notice that there is only a single underscore between self. and the name of the parent_class.

I also found Chat GPT's response to the following prompt helpful: _how do i inherit a private method from a base class in python?_ A problem to its response is that python doesn't have protected methods either. See https://stackoverflow.com/questions/11483366/protected-method-in-python

In [7]:
class colonDash_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"colonDash_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S2) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == ':': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == '-': next_state = self.S2
        else: next_state = self.Serr
        return next_state
    def S2(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.S2 # loop in state S2
        return next_state
    def Serr(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state

Let's run it and see if that works

In [8]:
my_FSA = colonDash_FSA()
input_string = ":-"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")


The  colonDash_FSA FSA accepted the input string ' :- '


***
Since that worked, let's define a second FSA, one that accepts any string that starts with ":"

In [12]:
class colon_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"colon_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S1) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == ':': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.S1 # loop in state S1
        return next_state
    def Serr(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state

... and run both it and the colon-dash FSA ...

In [15]:
my_colonDash_FSA = colonDash_FSA()
my_colon_FSA = colon_FSA()
input_string = ":ab"
accept_status_1 = my_colonDash_FSA.Run(input_string)
accept_status_2 = my_colon_FSA.Run(input_string)
if accept_status_1: print("The ", my_colonDash_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_colon_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
if accept_status_2: print("The ", my_colon_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_colon_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")


The  colon_FSA FSA did not accept the input string ' :ab '
The  colon_FSA FSA accepted the input string ' :ab '


... and one more time to see what happens on input strings that begin with ":-"

In [16]:
my_colonDash_FSA.Reset()
my_colon_FSA.Reset()
input_string = ":-ab"
accept_status_1 = my_colonDash_FSA.Run(input_string)
accept_status_2 = my_colon_FSA.Run(input_string)
if accept_status_1: print("The ", my_colonDash_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_colon_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
if accept_status_2: print("The ", my_colon_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_colon_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")


The  colonDash_FSA FSA accepted the input string ' :-ab '
The  colon_FSA FSA accepted the input string ' :-ab '
